Hypothesis: 

Short-term reversal is more pronounced in high-quality firms.
A short-term crash for a low-quality firm is more likely to be due to significant, catastrophic news.

Two motivating examples:

    1. a pharmaceutical company failing a medical trial (TRVN)
    2. a company committing fraud (Nikola)

If we are correct, filtering for quality prior to betting on mean-reversion should yield higher returns

In [53]:
import sf_quant.data as sfd
import sf_quant.optimizer as sfo
import sf_quant.backtester as sfb
import sf_quant.performance as sfp

In [54]:
import polars as pl
import datetime as dt
import yaml

In [55]:
all_columns = [
    "date", "rootid", "barrid", "issuerid", "instrument",
    "name", "cusip", "ticker", "price", "return", "specific_return",
    "market_cap", "price_source", "currency", "iso_country_code",
    "iso_currency_code", "yield", "total_risk", "specific_risk",
    "historical_beta", "predicted_beta", "russell_1000", "russell_2000",
    "daily_volume", "average_daily_volume_30", "average_daily_volume_60",
    "average_daily_volume_90", "bid_ask_spread", "average_daily_bid_ask_spread_30",
    "average_daily_bid_ask_spread_60", "average_daily_bid_ask_spread_90"
]

In [57]:
start = dt.date(2023, 1, 1)
end = dt.date(2024, 1, 31)

data = sfd.load_assets(
    start=start,
    end=end,
    in_universe=True,
    columns=all_columns
)
data = (
    data.with_columns(
        pl.col('return').truediv(100).alias('return')))

data

date,rootid,barrid,issuerid,instrument,name,cusip,ticker,price,return,specific_return,market_cap,price_source,currency,iso_country_code,iso_currency_code,yield,total_risk,specific_risk,historical_beta,predicted_beta,russell_1000,russell_2000,daily_volume,average_daily_volume_30,average_daily_volume_60,average_daily_volume_90,bid_ask_spread,average_daily_bid_ask_spread_30,average_daily_bid_ask_spread_60,average_daily_bid_ask_spread_90
date,str,str,str,str,str,str,str,f64,f64,f64,f64,str,str,str,str,f64,f64,f64,f64,f64,bool,bool,f64,f64,f64,f64,f64,f64,f64,f64
2023-01-03,"""USA06Z1""","""USA06Z1""","""A88843""","""STOCK""","""MIMEDX GROUP INC""","""602496101""","""MDXG""",2.97,0.068345,6.869,3.3759e8,"""MSCIBARRA""","""USD""","""USA""","""USD""",0.0,62.494865,47.65898,1.402132,1.43155,false,true,189178.0,246672.9,227155.18,217032.89,0.01,0.0135,0.01333,0.01328
2023-01-04,"""USA06Z1""","""USA06Z1""","""A88843""","""STOCK""","""MIMEDX GROUP INC""","""602496101""","""MDXG""",3.0,0.010101,-2.276,3.41004537e8,"""MSCIBARRA""","""USD""","""USA""","""USD""",0.0,62.526714,47.539847,1.401773,1.428854,false,true,235382.0,246271.65,227360.85,218255.8,0.01,0.0135,0.01325,0.01328
2023-01-05,"""USA06Z1""","""USA06Z1""","""A88843""","""STOCK""","""MIMEDX GROUP INC""","""602496101""","""MDXG""",3.08,0.026667,4.046,3.5010e8,"""MSCIBARRA""","""USD""","""USA""","""USD""",0.0,61.587914,47.755957,1.378388,1.37346,false,true,262605.0,248296.35,228220.46,218374.85,0.01,0.013,0.01317,0.01328
2023-01-06,"""USA06Z1""","""USA06Z1""","""A88843""","""STOCK""","""MIMEDX GROUP INC""","""602496101""","""MDXG""",3.21,0.042208,3.172,3.6487e8,"""MSCIBARRA""","""USD""","""USA""","""USD""",0.0,62.122262,48.110135,1.38589,1.389274,false,true,192226.0,251077.9,227838.68,217953.1,0.01,0.013,0.01293,0.01323
2023-01-09,"""USA06Z1""","""USA06Z1""","""A88843""","""STOCK""","""MIMEDX GROUP INC""","""602496101""","""MDXG""",3.28,0.021807,2.375,3.7283e8,"""MSCIBARRA""","""USD""","""USA""","""USD""",0.0,61.725551,48.612414,1.390513,1.348687,false,true,251390.0,262598.37,222707.49,220592.3,0.01,0.01263,0.01282,0.01328
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-01-25,"""USBPM41""","""USBPM41""","""138729""","""STOCK""","""WORTHINGTON STEEL INC""","""982104101""","""WS""",29.38,-0.003054,0.374,1.4761e9,"""MSCIBARRA""","""USD""","""USA""","""USD""",2.27758,62.699869,50.968819,1.09065,1.408389,false,true,50806.0,63945.85,99704.18,99704.18,0.05,0.0445,0.10184,0.10184
2024-01-26,"""USBPM41""","""USBPM41""","""138729""","""STOCK""","""WORTHINGTON STEEL INC""","""982104101""","""WS""",29.58,0.006807,-0.363,1.4861e9,"""MSCIBARRA""","""USD""","""USA""","""USD""",2.27758,62.561369,50.922137,1.085788,1.40225,false,true,49097.0,61568.0,98469.85,98469.85,0.04,0.0445,0.10026,0.10026
2024-01-29,"""USBPM41""","""USBPM41""","""138729""","""STOCK""","""WORTHINGTON STEEL INC""","""982104101""","""WS""",30.28,0.023665,0.383,1.5213e9,"""MSCIBARRA""","""USD""","""USA""","""USD""",2.27758,62.493405,50.93332,1.080155,1.395143,false,true,26090.0,60162.11,104143.36,96746.52,0.04,0.04526,0.06282,0.09875


In [58]:
# get log return
data = (
    data.with_columns(
        pl.col('return')
        .log1p()
        .alias('log_return')
    )
)

period = 21

# get short-term reversal scores
data = (
    data.with_columns(
        pl.col('log_return')
        .rolling_sum(window_size=period)
        .over('barrid')
        .mul(-1)
        .alias('short_term_reversal')
    )
)

In [59]:
# get alphas
data = (
    data.with_columns(
        ((pl.col("short_term_reversal") - pl.col("short_term_reversal").mean().over("date")) / pl.col("short_term_reversal").std().over("date")).alias("score")
    )
    .with_columns(
        (pl.col("score") / pl.col("specific_risk")).alias("alpha")
    )
    .with_columns(
        pl.col('specific_risk').truediv(100)
    )
    # .drop_nulls()
    .select(["barrid", "date", "price", "predicted_beta", "specific_risk", "short_term_reversal", "score", "alpha"])
)

In [ ]:
# alphas filter
data = data.drop_nulls('alpha')

: 

In [ ]:
constraints = [
    sfo.constraints.FullInvestment(),
    sfo.constraints.LongOnly(),
    sfo.constraints.NoBuyingOnMargin(),
    sfo.constraints.UnitBeta()
]
weights = sfb.backtest_sequential(
    data=data,
    constraints=constraints,
    gamma=10,
)

Running backtest:  12%|█▏        | 30/251 [19:14<2:19:07, 37.77s/it]

In [ ]:
returns = sfp.generate_returns_from_weights(weights=weights)

In [ ]:
portfolio_returns = sfp.generate_returns_chart(
    returns=returns,
    title="Mean-variance backtest",
    log_scale=True,
    file_name="mv_typical.png"
)

In [ ]:
summary = sfp.generate_summary_table(
        returns=returns
    )
summary

In [ ]:
# now we filter for quality and do the same backtest

# price > $5
data = data.with_columns(
    pl.col('price').shift(1).over('barrid').alias('price_lag')
)
data = data.filter(
    (pl.col('price_lag') >= (5))
).drop_nulls('short_term_reversal')


# high dollar volume
MIN_DOLLAR_VOLUME = 2_000_000
data = data.with_columns(
    pl.col("average_daily_volume_90").shift(1).over("barrid").alias("adv_90_lag")
)
data = data.with_columns(
    (pl.col("adv_90_lag") * pl.col("price_lag")).alias("avg_daily_dollar_volume")
)
data = data.filter(
    (pl.col('avg_daily_dollar_volume') >= MIN_DOLLAR_VOLUME)
)


# low spread
MAX_SPREAD_BPS = 0.10 
data = data.with_columns(
    pl.col("average_daily_bid_ask_spread_90").shift(1).over("barrid").alias("spread_90_lag")
)
data = data.filter(
    (pl.col('spread_90_lag') <= MAX_SPREAD_BPS)
)

# drop nulls
data = data.drop_nulls(["short_term_reversal", "avg_daily_dollar_volume", "spread_90_lag"])

In [ ]:
constraints = [
    sfo.constraints.FullInvestment(),
    sfo.constraints.LongOnly(),
    sfo.constraints.NoBuyingOnMargin(),
    sfo.constraints.UnitBeta()
]
weights = sfb.backtest_sequential(
    data=data,
    constraints=constraints,
    gamma=10,
)

In [ ]:
returns = sfp.generate_returns_from_weights(weights=weights)

In [ ]:
portfolio_returns = sfp.generate_returns_chart(
    returns=returns,
    title="Mean-variance backtest",
    log_scale=True,
    file_name="mv_quality.png"
)

In [ ]:
summary = sfp.generate_summary_table(
        returns=returns
    )
summary